# Forest Fire

In [1]:
import pathlib
import time
import IPython

import numpy as np

In [2]:
class CAForestFire:
    def __init__(
        self,
        shape: tuple[int, int],
        p: float,
        f: float,
        q: float,
        g: float = 1.0,
        chars: list[str] = [".", "T", "W"],
    ):
        self.state_ids = {
            "ashes": 0,
            "tree": 1,
            "fire": 2,
        }

        self.grid = np.full(shape, self.state_ids["ashes"], dtype=np.uint8)

        self.p = p  # rate of spontaneous growth
        self.f = f  # rate of spontaneous fire
        self.q = q  # rate of induced growth
        self.g = g  # rate of induced fire

        self.chars = np.array(chars)

    def update_grid(self):
        grid_new = np.copy(self.grid)

        is_ashes = self.grid == self.state_ids["ashes"]
        is_tree = self.grid == self.state_ids["tree"]
        is_fire = self.grid == self.state_ids["fire"]

        rate_spontaneous_growth = np.random.random(self.grid.shape) < self.p
        rate_spontaneous_fire = np.random.random(self.grid.shape) < self.f
        rate_induced_growth = np.random.random(self.grid.shape) < self.q
        rate_induced_fire = np.random.random(self.grid.shape) < self.g

        # Avoid for loops and such.
        has_tree_above = np.roll(is_tree, 1, axis=0)
        has_tree_under = np.roll(is_tree, -1, axis=0)
        has_tree_left = np.roll(is_tree, 1, axis=1)
        has_tree_right = np.roll(is_tree, -1, axis=1)

        has_fire_above = np.roll(is_fire, 1, axis=0)
        has_fire_under = np.roll(is_fire, -1, axis=0)
        has_fire_left = np.roll(is_fire, 1, axis=1)
        has_fire_right = np.roll(is_fire, -1, axis=1)

        has_tree_neighbor = has_tree_above | has_tree_under | has_tree_left | has_tree_right
        has_fire_neighbor = has_fire_above | has_fire_under | has_fire_left | has_fire_right

        grid_new[is_ashes & rate_spontaneous_growth] = self.state_ids["tree"]
        grid_new[is_tree & rate_spontaneous_fire] = self.state_ids["fire"]
        grid_new[is_ashes & rate_induced_growth & has_tree_neighbor] = self.state_ids["tree"]
        grid_new[is_tree & rate_induced_fire & has_fire_neighbor] = self.state_ids["fire"]
        grid_new[is_fire] = self.state_ids["ashes"]

        self.grid = grid_new

    def __str__(self):
        grid_chars = self.chars[self.grid]
        s = ""
        for row in grid_chars:
            s += " ".join(row)
            s += "\n"
        return s

    def save_to_file(self, counts_state):
        try:
            filename = f"{pathlib.Path(__vsc_ipynb_file__).stem}_counts.txt"
        except:
            filename = "forest_fire_counts.txt"
        with open(filename, "w") as f:
            for count_state in counts_state:
                count_ashes, count_tree, count_fire = count_state
                f.write(f"{count_ashes} {count_tree} {count_fire}\n")

    def count_state(self):
        is_ashes = self.grid == self.state_ids["ashes"]
        is_tree = self.grid == self.state_ids["tree"]
        is_fire = self.grid == self.state_ids["fire"]

        count_ashes = np.sum(is_ashes)
        count_tree = np.sum(is_tree)
        count_fire = np.sum(is_fire)

        count_state = (count_ashes, count_tree, count_fire)
        return count_state

    def run(self, steps: int, seconds_per_step: float = 0.1):
        counts_state = []
        print(self)

        for t in range(steps + 1):
            count_state = self.count_state()
            counts_state.append(count_state)

            IPython.display.clear_output(wait=True)
            time.sleep(seconds_per_step)
            self.update_grid()
            print(self)

        self.save_to_file(counts_state)

In [3]:
ca_forest_fire = CAForestFire(
    shape=(100, 80),
    p=0.3,
    f=0.01,
    q=0.5,
    g=1.0,
    chars=[".", "T", "W"],
)

ca_forest_fire.run(steps=200)

. . . T W W T T T W . W W T T . T . W T T T W . . T T W . . T . W W T W T W . . W T . W . . T T W . T . W . W . W . . . W W T T . . T W T W . . W . T . T T . W T T
W . . . . T T T W . W W T T . T T W T W T T T . T . . T . W T W W . W . T T T . T T T T T . T W . . . W T W . T . W . . . . T W . . T T T W W T T W T . T T W . W T
W . T . T W T T T . . . . T . . T . T . T T T T T W W . T T T W . . . W T W . T W . T T T . . W . W W T T T T T W . T T . . T . T W T T T . T T . T W . W T T T T T
. T T T . . T T W . W W T T . . . W . W . W T W W . T T W T . . T . . T T T . . T W T T T . . . T W . W T T . T T T W T T W T . W T T . T . T . T T T W T T W . T W
W . T T T . . T T W T T W . W . T T W W . . . . . . . W . W . W . T T T . T . . . W T T T . . T W . . T W . . W . . . T T . . . T T . T W . W W T . T T T W . . T T
T T T T W . . . T . T T T W T W . W . . T . T . T T W T T . W . T . W . T . . T . W T T . T W W . T T T T T . T W T W . T . . T T W . W . W T . . . T W . T W T T T
T . T T T . T W 